Download data from https://www.kaggle.com/competitions/nlp-ucu-2025-task-1/data into this folder.

You need train.json and test.json to start.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./sample_solution_local.ipynb
./sample_solution_kaggle.ipynb
./test.json
./README.md
./train.json


In [2]:
train_df = pd.read_json("train.json", lines=True)
test_df = pd.read_json("test.json", lines=True)

target_chats = set("абвгґдеєжзиіїйклмнопрстуфхцчшщьюя' ") # remove emails, urls, phones

def normalize_text(text):
    # leave only allowed characters
    # replace with a space, then remove extra spaces
    text = text.lower()
    for char in text:
        if char.lower() not in target_chats:
            text = text.replace(char, " ")

    text = " ".join(text.split())
    return text
train_df["text"] = train_df["text"].apply(normalize_text)
test_df["text"] = test_df["text"].apply(normalize_text)
train_df


,text,target
0,приобрела всю серию детских судоку после того ...,5
1,книга детектив мейзі хітчінз або справа про ру...,5
2,коли придбала книжку то була певна що історія ...,5
3,книжка брехун серед нас виконана в жанрі реалі...,4
4,ця книга чудовий екземпляр подарунку іноземцям...,5
...,...,...
55399,детям интересно будет поближе познакомиться с ...,5
55400,то сборник новелл в жанре литератур ужасов нов...,5
55401,бернар вербер не в бирает легких тем его книги...,5
55402,що не кажіть а осінь це мабуть найсприятливіша...,5


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Create document-term matrix using word counts
count_vectorizer = CountVectorizer(max_features=1000)
X_train = count_vectorizer.fit_transform(train_df["text"])
X_test = count_vectorizer.transform(test_df["text"])

# Get training and test labels
y_train = train_df['target']


# Initialize and train the MultinomialNB classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Make predictions
y_pred = nb_classifier.predict(X_test)
y_pred

array([5, 1, 5, ..., 5, 5, 5])

In [4]:
test_df = test_df.drop(columns=["text"])

test_df["target"] = y_pred
test_df

,id,target
0,0,5
1,1,1
2,2,5
3,3,5
4,4,5
...,...,...
13847,13847,3
13848,13848,3
13849,13849,5
13850,13850,5


In [5]:
test_df.to_csv("submission.csv", index=False)